Production Implementation: The "Instructor" Pattern
Here is a Python implementation of an Enterprise-Grade Validated Wrapper. This code guarantees the output is not just JSON, but valid JSON that adheres to specific logic (e.g., no negative prices).

Prerequisites: pip install instructor pydantic openai

In [ ]:
import instructor
from pydantic import BaseModel, Field, field_validator, ValidationInfo
from openai import OpenAI
from typing import List

# 1. Initialize the Wrapped Client
# This "patches" the OpenAI client to support response_model
client = instructor.from_openai(OpenAI())

# 2. Define the Schema (The Contract)
# We use Pydantic for strong typing and validation logic.
class ProductExtraction(BaseModel):
    product_name: str = Field(..., description="The clear name of the product")
    price: float = Field(..., description="Price extracted from text")
    currency: str = Field(..., description="ISO Currency code (USD, EUR)")
    category: str = Field(..., description="High-level category")

    # 3. Built-in Validation Logic (The "Guardrail")
    # This runs POST-generation but PRE-return to your app.
    @field_validator('price')
    def price_must_be_positive(cls, v: float, info: ValidationInfo):
        if v < 0:
            raise ValueError(f"Price cannot be negative: {v}")
        return v
    
    @field_validator('currency')
    def validate_currency(cls, v: str):
        if v not in ['USD', 'EUR', 'INR']:
            raise ValueError(f"Unsupported currency: {v}. Convert to USD.")
        return v

# 4. The Wrapper Execution with Auto-Retries
# If the model hallucinates a negative price, Pydantic raises an error.
# Instructor catches the error, feeds it back to the LLM ("You generated -10, please fix"),
# and retries up to `max_retries`.
def extract_data_safely(text_input: str):
    try:
        resp = client.chat.completions.create(
            model="gpt-4o",
            response_model=ProductExtraction,
            max_retries=3,  # Self-Correction Loop
            messages=[
                {"role": "user", "content": f"Extract data: {text_input}"},
            ],
        )
        return resp.model_dump_json(indent=2)
    except Exception as e:
        # Fallback mechanism for Enterprise reliability
        return f"Error: Failed to extract valid data after retries. {e}"

# Usage
raw_text = "I bought a premium GamerChair for -200 dollars." 
# The model will likely try to output -200, fail validation, 
# receive the error, and correct it (e.g., absolute value or flag error).
print(extract_data_safely(raw_text))

SchemaFirstExtractor (best for “LLM → pipeline”)

Use case: incident extraction, contract parsing, ticket creation fields.
Mechanism: JSON Schema + constrained decode + schema validation.

NIM recommends using guided_json (JSON schema) for structured outputs for downstream processing reliability. 
NVIDIA Docs
+1

Enterprise code skeleton (NIM / OpenAI-compatible)

In [ ]:
import json
from typing import Any, Dict, Optional
from pydantic import BaseModel, Field, ValidationError
from openai import OpenAI


class Incident(BaseModel):
    severity: str = Field(pattern=r"^P[0-3]$")
    service: str
    root_cause: str
    action_items: list[str]


INCIDENT_SCHEMA: Dict[str, Any] = Incident.model_json_schema()


class SchemaFirstExtractor:
    def __init__(self, base_url: str, api_key: str, model: str):
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model

    def extract(self, text: str, max_retries: int = 2) -> Incident:
        sys = "Extract an incident report. Return JSON only. No extra keys."
        user = f"<TEXT>\n{text}\n</TEXT>"

        last_err: Optional[str] = None
        for _ in range(max_retries + 1):
            resp = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "system", "content": sys},
                          {"role": "user", "content": user}],
                # NIM structured generation:
                extra_body={"guided_json": INCIDENT_SCHEMA},
                temperature=0.2,
                max_tokens=400,
            )
            raw = resp.choices[0].message.content.strip()

            try:
                data = json.loads(raw)
                return Incident.model_validate(data)
            except (json.JSONDecodeError, ValidationError) as e:
                last_err = str(e)
                # Repair hint (still constrained by schema on next attempt)
                user = f"{user}\n\nThe previous JSON was invalid: {last_err}\nFix it."

        raise ValueError(f"Extraction failed after retries: {last_err}")


ConstrainedChoiceClassifier (zero hallucination labels)

Use case: route to tool, pick severity, choose “YES/NO”, choose category.
Mechanism: constrain output to exactly one from a list (guided_choice).

NIM supports constraining output to specific choices.

In [ ]:
choices = ["CREATE_TICKET", "ANSWER_FROM_KB", "ASK_CLARIFYING_QUESTION"]

resp = client.chat.completions.create(
    model=model,
    messages=[...],
    extra_body={"guided_choice": choices},
    temperature=0.0,
    max_tokens=20
)
route = resp.choices[0].message.content.strip()
